## 用Keras构建模型，双向双层LSTM
## Two-way double layer LSTM (Keras framework)

In [1]:
import argparse
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, Lambda, LSTM, TimeDistributed, Masking, \
    Bidirectional
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model, load_model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import os, pickle
import numpy as np

Using TensorFlow backend.


In [2]:
# 使用GPU训练
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### build model

In [3]:
data_all = pickle.load(open("../models/data_word_id_map.pkl","rb"))
train_x, train_y, test_x, test_y, id2word, word2id = data_all[0], data_all[1], data_all[2], data_all[3], data_all[4], data_all[5]
wordNum = len(id2word)

In [4]:
leng = train_x.shape[1]
inputs = Input(shape=(leng,wordNum), dtype='float32')
# 双向LSTM
lstm = Bidirectional(LSTM(300, activation='relu', return_sequences=True, dropout=0.3))(inputs)
lstm = Bidirectional(LSTM(300, activation='relu', return_sequences=False, dropout=0.3), name="utter")(lstm)
classes = 5
output = Dense(classes, activation='softmax')(lstm)

In [5]:
model = Model(inputs, output)
model.compile(optimizer='adadelta', loss='categorical_crossentropy')

In [9]:
epochs = 150
batch_size = 100
checkpoint = ModelCheckpoint('../models/emotion_recognition_model.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=50)

In [11]:
model.fit(train_x, train_y,
                  epochs=epochs,
                  batch_size=batch_size,
          callbacks=[early_stopping, checkpoint],
                  shuffle=True,
         validation_data=(test_x, test_y))

Train on 5904 samples, validate on 1476 samples
Epoch 1/150
5904/5904 [==============================] - 104s 18ms/step - loss: 1.5879 - val_loss: 1.5942

Epoch 00001: val_loss improved from inf to 1.59423, saving model to ./model_all_3.hdf5
Epoch 2/150
5904/5904 [==============================] - 100s 17ms/step - loss: 1.5833 - val_loss: 1.6053

Epoch 00002: val_loss did not improve from 1.59423
Epoch 3/150
5904/5904 [==============================] - 100s 17ms/step - loss: 1.5817 - val_loss: 1.5952

Epoch 00003: val_loss did not improve from 1.59423
Epoch 4/150
5904/5904 [==============================] - 100s 17ms/step - loss: 1.5787 - val_loss: 1.6012

Epoch 00004: val_loss did not improve from 1.59423
Epoch 5/150
5904/5904 [==============================] - 102s 17ms/step - loss: 1.5737 - val_loss: 1.5860

Epoch 00005: val_loss improved from 1.59423 to 1.58601, saving model to ./model_all_3.hdf5
Epoch 6/150
5904/5904 [==============================] - 100s 17ms/step - loss: 1.5672

5904/5904 [==============================] - 100s 17ms/step - loss: 0.7230 - val_loss: 1.2286

Epoch 00049: val_loss did not improve from 1.14652
Epoch 50/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.7106 - val_loss: 1.3512

Epoch 00050: val_loss did not improve from 1.14652
Epoch 51/150
5904/5904 [==============================] - 99s 17ms/step - loss: 0.7088 - val_loss: 1.8954

Epoch 00051: val_loss did not improve from 1.14652
Epoch 52/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.6909 - val_loss: 1.3047

Epoch 00052: val_loss did not improve from 1.14652
Epoch 53/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.6909 - val_loss: 1.2046

Epoch 00053: val_loss did not improve from 1.14652
Epoch 54/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.6841 - val_loss: 1.5241

Epoch 00054: val_loss did not improve from 1.14652
Epoch 55/150
5904/5904 [==============================] - 10


Epoch 00100: val_loss did not improve from 1.14652
Epoch 101/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.4776 - val_loss: 1.4077

Epoch 00101: val_loss did not improve from 1.14652
Epoch 102/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.4647 - val_loss: 1.3862

Epoch 00102: val_loss did not improve from 1.14652
Epoch 103/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.4739 - val_loss: 1.4352

Epoch 00103: val_loss did not improve from 1.14652
Epoch 104/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.4792 - val_loss: 1.4626

Epoch 00104: val_loss did not improve from 1.14652
Epoch 105/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.4730 - val_loss: 1.3830

Epoch 00105: val_loss did not improve from 1.14652
Epoch 106/150
5904/5904 [==============================] - 100s 17ms/step - loss: 0.4728 - val_loss: 1.4005

Epoch 00106: val_loss did not improve f

In [12]:
# from numba import cuda
# cuda.select_device(0)
# cuda.close()